In this notebook I am gonna work on the plots generated for the poster. Step:

- Aggregate the data in three different arrays:
    - total latency
    - tracking latency
    - display latency
    
- plto the figures